In [1]:
import os
import math
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, Dataset
import lightning as L
import finalnlp
from finalnlp.gpt.gpt_model import GPT
from finalnlp.gpt.nb.share import LitGPT, SortDataset, eval_split, length, num_digits, train_dataset, test_dataset, model_type, vocab_size, block_size, val_check_interval, max_steps
from finalnlp.replacer import replace_linears_in_pytorch_model
from finalnlp import bitnet1
from finalnlp import bitnet158
from lightning.pytorch import loggers as pl_loggers
%load_ext autoreload

Seed set to 42
Seed set to 42


In [2]:
x, y = train_dataset[0]
for a, b in zip(x,y):
    print(int(a),int(b))

2 -1
3 -1
0 -1
2 -1
2 -1
3 -1
0 -1
0 0
0 0
0 0
0 2
2 2
2 2
2 3
3 3


## Plain Linear Model

In [3]:
# autoencoder = LitAutoEncoder(Encoder(), Decoder())
L.seed_everything(42, workers=True)
model_config = GPT.get_default_config()
model_config.model_type = model_type
model_config.vocab_size = vocab_size
model_config.block_size = block_size
model = LitGPT(model_config, train_dataset.length)

wandb_logger = pl_loggers.WandbLogger("GPT-Sort-Problem-len" + str(length) + str(model_type))
wandb_logger.experiment.config.update(model_config)
wandb_logger.experiment.config.update({"problem": "sort", "linear_replacer": "none"})
wandb_logger.experiment.config.update({"length": length, "num_digits": num_digits})

trainer = L.Trainer(
    # callbacks=[EarlyStopping(monitor="train_loss", mode="min")],
    logger=wandb_logger,
    max_steps=max_steps,
    val_check_interval=val_check_interval,
)
wandb_logger.watch(model)
torch.set_float32_matmul_precision('medium')
trainer.fit(
    model=model,
    train_dataloaders=DataLoader(train_dataset, num_workers=15),
    val_dataloaders=DataLoader(test_dataset, num_workers=15),    
)

Seed set to 42


number of parameters: 0.09M


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: candrewlee14 (andrews-org). Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | GPT  | 86.0 K
-------------------------------
86.0 K    Trainable params
0         Non-trainable params
86.0 K    Total params
0.344     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=10000` reached.


In [4]:
trainer.validate(model=model, dataloaders=DataLoader(test_dataset, num_workers=15))

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_acc             0.972000002861023
        val_loss           0.031101757660508156
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 0.031101757660508156, 'val_acc': 0.972000002861023}]

In [5]:
# run a lot of examples from both train and test through the model and verify the output correctness
with torch.no_grad():
    train_score = eval_split(model, trainer, train_dataset, test_dataset, 'train', max_batches=50)
    test_score  = eval_split(model, trainer, train_dataset, test_dataset, 'test',  max_batches=50)

GPT claims that [3, 1, 1, 3, 2, 2, 3, 3] sorted is [1, 1, 2, 2, 2, 3, 3, 3] but gt is [1, 1, 2, 2, 3, 3, 3, 3]
GPT claims that [3, 3, 0, 2, 2, 3, 3, 1] sorted is [0, 1, 2, 2, 2, 3, 3, 3] but gt is [0, 1, 2, 2, 3, 3, 3, 3]
GPT claims that [3, 2, 2, 3, 3, 3, 3, 1] sorted is [1, 2, 2, 2, 3, 3, 3, 3] but gt is [1, 2, 2, 3, 3, 3, 3, 3]
train final score: 947/1000 = 94.70% correct
GPT claims that [3, 2, 3, 3, 1, 2, 2, 3] sorted is [1, 2, 2, 2, 2, 3, 3, 3] but gt is [1, 2, 2, 2, 3, 3, 3, 3]
GPT claims that [3, 3, 1, 2, 0, 2, 3, 3] sorted is [0, 1, 2, 2, 2, 3, 3, 3] but gt is [0, 1, 2, 2, 3, 3, 3, 3]
GPT claims that [0, 0, 2, 0, 0, 0, 1, 3] sorted is [0, 0, 0, 0, 1, 2, 2, 3] but gt is [0, 0, 0, 0, 0, 1, 2, 3]
test final score: 239/250 = 95.60% correct


In [6]:
# let's run a random given sequence through the model as well
n = train_dataset.length
inp = torch.tensor([[0, 0, 2, 1, 0, 1, 3, 4, 5, 6]], dtype=torch.long)
inp, y = train_dataset[0]
inp = inp.unsqueeze(dim=0)
# assert inp[0].nelement() == n
with torch.no_grad():
    cat = model.model.generate(inp, n, do_sample=False)
sol = torch.sort(inp[0])[0]
sol_candidate = cat[:, n:]
print('input sequence  :', inp.tolist())
print('predicted sorted:', sol_candidate.tolist())
print('gt sort         :', sol.tolist())
print('matches         :', bool((sol == sol_candidate).all()))

input sequence  : [[1, 3, 3, 0, 2, 1, 3, 2, 0, 1, 1, 2, 2, 3, 3]]
predicted sorted: [[0, 1, 1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]]
gt sort         : [0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3]
matches         : False
